# Generate kerchunk sidecar file for Surface Air Temperature data in NetCDF formats (freq: daily and monthly)

In [1]:
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
# import scipy as sp
import xarray as xr
import fsspec
from kerchunk.hdf import SingleHdf5ToZarr
from pathlib import Path
import ujson
import intake_esm
import intake
import fsspec
import kerchunk.hdf
from kerchunk.combine import MultiZarrToZarr

In [2]:
import dask
from dask.distributed import Client, performance_report
from dask_jobqueue import PBSCluster

In [3]:
import os
import ujson
from fsspec.implementations.local import LocalFileSystem
import kerchunk.hdf
import pandas as pd

In [4]:
# File paths
rda_scratch = "/gpfs/csfs1/collections/rda/scratch/harshah"
rda_data    = "/gpfs/csfs1/collections/rda/data/"
myrda_data  = rda_data + 'harshah/'
era5_path   = rda_data + "ds633.0/e5.oper.an.sfc/"
#
pelican_test = '/glade/campaign/collections/rda/transfer/harshah/pelican_test'
zarr_path   = rda_scratch + "/tas_zarr/"

In [5]:
def gen_json(file_url, write_json=False):
    print(f'generating {file_url}')
    with fs.open(file_url, **so) as infile:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(infile, file_url, inline_threshold=366 )
        year = file_url.split('/')[-1].split('.')[0]
        file_basename = os.path.basename(file_url)
        outfile = f'{file_basename}.json'
        if write_json:
            with fs.open(outfile, 'wb') as f:
                print(f'writing {outfile}')
                f.write(ujson.dumps(h5chunks.translate()).encode());
        return h5chunks.translate()

In [6]:
## Find NetCDF files with tas (Surface air temperature at 2m) using glob and a search pattern
tas_pattern = era5_path + "**/e5.oper.an.sfc.128_167_2t.*.nc"
tas_ncfiles = glob.glob(tas_pattern, recursive=True)

In [7]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-wk24-hpc',
    cores = 1,
    memory = '8GiB',
    processes = 1,
    local_directory = rda_scratch+'/dask/spill',
    log_directory = rda_scratch +'/dask/',
    resource_spec = 'select=1:ncpus=1:mem=8GB',
    queue = 'casper',
    walltime = '2:00:00',
    #interface = 'ib0'
    interface = 'ext'
)

/glade/work/harshah/conda-envs/arco_experiments/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37213 instead
  warnings.warn(


In [8]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/37213/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/37213/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.96:33717,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/37213/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [9]:
cluster.scale(10)

### Open intake catalog and extract paths

In [10]:
catalog = intake.open_esm_datastore(rda_scratch + '/intake_catalogs/posix/era5_catalog.json')
catalog

/glade/work/harshah/conda-envs/arco_experiments/lib/python3.12/site-packages/intake_esm/cat.py:249: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,unique
era_id,1
datatype,2
level_type,1
step_type,7
table_code,4
param_code,164
variable,212
long_name,212
units,33
year,85


In [11]:
cat_temp = catalog.search(variable ='VAR_2T')
cat_temp

,unique
era_id,1
datatype,1
level_type,0
step_type,1
table_code,1
param_code,1
variable,1
long_name,1
units,1
year,85


In [12]:
cat_temp.df['path'].head()

0    https://data.rda.ucar.edu/ds633.0/e5.oper.an.s...
1    https://data.rda.ucar.edu/ds633.0/e5.oper.an.s...
2    https://data.rda.ucar.edu/ds633.0/e5.oper.an.s...
3    https://data.rda.ucar.edu/ds633.0/e5.oper.an.s...
4    https://data.rda.ucar.edu/ds633.0/e5.oper.an.s...
Name: path, dtype: object

### Read these netcdf files and rewrite them with uniform chunk size

In [13]:
# fs = LocalFileSystem()
so = dict(mode='rb', default_fill_cache=False, default_cache_type='first')

In [14]:
# def rechunk_netcdf(file_url, output_directory, chunk_size):
#     """Rechunk a NetCDF file and save it to a new directory with the same file name, removing '#mode=bytes'."""
#     # Open the dataset with the specified chunk size
#     ds = xr.open_dataset(file_url, chunks=chunk_size)
    
#     # Extract the file name from the file URL and remove '#mode=bytes' if present
#     file_name = os.path.basename(file_url).replace('#mode=bytes', '')
    
#     # Create the output path
#     output_path = os.path.join(output_directory, file_name)
    
#     # Save the rechunked dataset to the new directory
#     ds.to_netcdf(output_path)
#     print(f'Saved rechunked file to {output_path}')
#     return output_path

# def process_and_rechunk_files(file_urls, output_directory, chunk_size):
#     """Process a list of file URLs, rechunk them, and save them to a new directory."""
#     os.makedirs(output_directory, exist_ok=True)
#     rechunked_files = []
#     for file_url in file_urls:
#         rechunked_file = rechunk_netcdf(file_url, output_directory, chunk_size)
#         rechunked_files.append(rechunked_file)
#     return rechunked_files

In [15]:
def rechunk_netcdf(file_url, output_directory, chunk_size, overwrite=False):
    """Rechunk a NetCDF file and save it to a new directory with the same file name, removing '#mode=bytes'.
    Args:
        file_url (str): URL or path to the input NetCDF file.
        output_directory (str): Directory where the rechunked file will be saved.
        chunk_size (dict): Dictionary specifying the chunk sizes.
        overwrite (bool): Whether to overwrite existing files. Default is False.
    """
    # Extract the file name from the file URL and remove '#mode=bytes' if present
    file_name = os.path.basename(file_url).replace('#mode=bytes', '')
    
    # Create the output path
    output_path = os.path.join(output_directory, file_name)
    
    # Check if the file already exists and overwrite is False
    if os.path.exists(output_path) and not overwrite:
        print(f'File {output_path} already exists and overwrite is set to False. Skipping.')
        return output_path
    
    # Open the dataset with the specified chunk size
    ds = xr.open_dataset(file_url, chunks=chunk_size)
    
    # Save the rechunked dataset to the new directory
    ds.to_netcdf(output_path)
    print(f'Saved rechunked file to {output_path}')
    return output_path

def process_and_rechunk_files(file_urls, output_directory, chunk_size, overwrite=False):
    """Process a list of file URLs, rechunk them, and save them to a new directory.
    Args:
        file_urls (list): List of URLs or paths to the input NetCDF files.
        output_directory (str): Directory where the rechunked files will be saved.
        chunk_size (dict): Dictionary specifying the chunk sizes.
        overwrite (bool): Whether to overwrite existing files. Default is False.
    """
    os.makedirs(output_directory, exist_ok=True)
    rechunked_files = []
    for file_url in file_urls:
        rechunked_file = rechunk_netcdf(file_url, output_directory, chunk_size, overwrite)
        rechunked_files.append(rechunked_file)
    return rechunked_files

In [16]:
def gen_individual_json(file_url, output_directory='.', inline_threshold=366):
    """Generate individual Kerchunk JSON sidecar file for a given file URL."""
    if file_url.startswith("http"):
        so = dict(mode='rb')
    else:
        so = dict(mode='rb', default_fill_cache=False, default_cache_type='first')
    
    with fsspec.open(file_url, **so) as infile:
        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(infile, file_url, inline_threshold=inline_threshold)
        file_basename = os.path.basename(file_url)
        outfile = os.path.join(output_directory, f'{file_basename}.json')
        with open(outfile, 'w') as f:
            ujson.dump(h5chunks.translate(), f)
        return outfile

def process_files(file_paths, output_directory='.'):
    """Process a list of file paths and generate individual JSON sidecar files."""
    json_files = []
    for file_path in file_paths:
        json_file = gen_individual_json(file_path, output_directory)
        json_files.append(json_file)
    return json_files

def combine_json_files(json_files, output_path, concat_dim='time'):
    """Combine individual JSON sidecar files into a single aggregate JSON sidecar file."""
    refs = [ujson.load(open(f)) for f in json_files]

    print('Combining references into a single JSON sidecar file')
    mzz = MultiZarrToZarr(
        path=json_files,
        indicts=refs, 
        concat_dims=[concat_dim],
    )
    combined = mzz.translate()
    
    with open(output_path, 'w') as f:
        ujson.dump(combined, f)
    print(f'Combined JSON sidecar file written to {output_path}')

In [17]:
%%time
# Example usage in Jupyter Notebook
# Assuming `catalog` is your intake catalog and has a DataFrame attribute `df` with a 'path' column

cat_temp_df      = cat_temp.df  # This should be your actual DataFrame

CPU times: user 23 μs, sys: 5 μs, total: 28 μs
Wall time: 31.7 μs


In [ ]:
%%time
output_directory = myrda_data +'/era5_tas'  # Update to your actual path
chunk_size = {"time": 27, "lat": 139, "lon": 277}  # Update with your desired chunk sizes
# Rechunk the files and save them to the new directory
file_urls = cat_temp_df['path'][:36].tolist()
rechunked_files = process_and_rechunk_files(file_urls, output_directory, chunk_size)

File /gpfs/csfs1/collections/rda/data/harshah//era5_tas/e5.oper.an.sfc.128_167_2t.ll025sc.1940010100_1940013123.nc already exists and overwrite is set to False. Skipping.
File /gpfs/csfs1/collections/rda/data/harshah//era5_tas/e5.oper.an.sfc.128_167_2t.ll025sc.1940020100_1940022923.nc already exists and overwrite is set to False. Skipping.
File /gpfs/csfs1/collections/rda/data/harshah//era5_tas/e5.oper.an.sfc.128_167_2t.ll025sc.1940030100_1940033123.nc already exists and overwrite is set to False. Skipping.
File /gpfs/csfs1/collections/rda/data/harshah//era5_tas/e5.oper.an.sfc.128_167_2t.ll025sc.1940040100_1940043023.nc already exists and overwrite is set to False. Skipping.
File /gpfs/csfs1/collections/rda/data/harshah//era5_tas/e5.oper.an.sfc.128_167_2t.ll025sc.1940050100_1940053123.nc already exists and overwrite is set to False. Skipping.
File /gpfs/csfs1/collections/rda/data/harshah//era5_tas/e5.oper.an.sfc.128_167_2t.ll025sc.1940060100_1940063023.nc already exists and overwrite i

/glade/work/harshah/conda-envs/arco_experiments/lib/python3.12/site-packages/xarray/core/dataset.py:274: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 27. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [ ]:
def create_new_file_paths(file_urls, output_directory):
    """Create new file paths for the rechunked files."""
    new_file_paths = []
    for file_url in file_urls:
        # Extract the file name from the file URL and remove '#mode=bytes'
        file_name = os.path.basename(file_url).replace('#mode=bytes', '')
        # Create the new file path
        new_file_path = os.path.join(output_directory, file_name)
        new_file_paths.append(new_file_path)
    return new_file_paths

# Create new file paths for the rechunked files
new_file_paths = create_new_file_paths(file_urls, output_directory)

In [ ]:
sidecar_directory = rda_scratch + '/sidecar/era5/'
sidecar_path      = sidecar_directory + 'era5_2t_https.json'  # Update to your desired output directory
# Generate individual JSON sidecar files
json_files = process_files(new_file_paths, sidecar_directory)

In [39]:
%%time
# Combine individual JSON sidecar files into a single aggregate JSON sidecar file
combine_json_files(json_files, output_path)

Combining references into a single JSON sidecar file


ValueError: Found chunk size mismatch:
                        at prefix VAR_2T in iteration 1 (file /gpfs/csfs1/collections/rda/scratch/harshah/sidecar/era5/e5.oper.an.sfc.128_167_2t.ll025sc.1940050100_1940053123.nc#mode=bytes.json)
                        new chunk: [27, 139, 279]
                        chunks so far: [27, 139, 277]

In [ ]:
# # Path to the combined Kerchunk sidecar file
# combined_kerchunk_path =  output_path # Update to your actual path

# # Create a fsspec filesystem using the Kerchunk reference file
# fs = fsspec.filesystem(
#     "reference", fo=combined_kerchunk_path, remote_protocol="https"
# )

# # Get a mapper from the filesystem
# m = fs.get_mapper("")

# # Open the dataset using xarray
# ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False))

# # Print the dataset
# print(ds)

####################################

In [23]:
# # Example usage in Jupyter Notebook
# # Assuming `catalog` is your intake catalog and has a DataFrame attribute `df` with a 'path' column

# cat_temp_df = cat_temp.df  # This should be your actual DataFrame
# file_paths = cat_temp_df['path'].tolist()
# output_directory = rda_scratch + '/sidecar/era5/'  # Update to your desired output directory

# # Process the files and generate JSON sidecar files
# process_files(file_paths, output_directory)

## Let us rewrite these files into a zarr store